In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data, io
from skimage.color import label2rgb
import skimage
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import cv2

!ls ../results

labels_list_file = r"/home/anhaoran/data/pedestrian_attributes_PETA/PETA/labels.txt" 
labels_list_data = open(labels_list_file)
lines = labels_list_data.readlines()
attributes_list = []
for line in lines:
    data = line.split()
    #print(attr)
    attributes_list.append(data[1])
print("The total attributes labeled: ", len(attributes_list))
print(attributes_list)

filename = r"../results/PETA.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 61))
for i in range(length):
    data_y[i] = np.array(data[i, 1:1+61], dtype="float32")
y_test = data_y#[11400:]
print("The shape of the y_test is: ", y_test.shape)

1.jpg
berk.csv
berk-test.csv
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv
The total attributes labeled:  65
['accessoryHeadphone', 'personalLess15', 'personalLess30', 'personalLess45', 'personalLess60', 'personalLarger60', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'lowerBodyCapri', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'lowerBodyCasual', 'upperBodyCasual', 'personalFemale', 'carryingFolder', 'lowerBodyFormal', 'upperBodyFormal', 'accessoryHairBand', 'accessoryHat

# LBP

In [2]:
data_x = []
radius = 3
n_points = 8 * radius
for i in range(length):
    image = cv2.imread(data[i, 0])
    image = cv2.resize(image,(160, 75))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, n_points, radius)
    #print(lbp.shape)#(75, 160)
    data_x.append(np.reshape(lbp, (-1,)))
data_x = np.asarray(data_x)
print(data_x.shape)

(19000, 12000)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, Y_train ,Y_test = train_test_split(data_x, data_y, test_size=0.2)
neigh_lbp = KNeighborsClassifier(n_neighbors=5)
neigh_lbp.fit(X_train, Y_train)
#print(neigh_lbp.score(X_test, Y_test))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [4]:
pred_lbp = neigh_lbp.predict(X_test)

In [12]:
def mA(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true[:])
    N = M - P
    TP = sum(y_pred[:]*y_true[:])
    TN = list(y_pred[:]+y_true[:] == 0).count(True)
    if P != 0:
        res = TP/P + TN/N
    else:
        res = TN/N
    return res / 2

diction_lbp = {}
for i in range(61):
    print(mA(Y_test[:, i], pred_lbp[:, i]))#attributes_list[i], ': ', 
    diction_lbp.update({attributes_list[i]:mA(Y_test[:, i], pred_lbp[:, i])})

0.9611423696451423
0.9308863394326653
0.9123460030603925
0.9086739378161278
0.9060473276349486
0.9263244041466706
0.9597920886206754
0.9372817844111028
0.8966085623876779
0.943665850146572
0.920957156251274
0.918079387780443
0.999736564805058
0.9996039081066808
0.9004746998095113
0.9048059964726631
0.9105471195634551
0.8765147867260543
0.8992844436028884
0.90240480871962
0.9596663029427339
0.9216369651853522
0.961361254473908
0.8943486577460289
0.8989596863204239
0.9697021438057294
0.9153768976186548
0.9268927292090232
0.905479380485766
0.9131139008170506
0.9080740127544367
0.4994736842105263
0.4998684210526316
0.9761427864304544
0.9110661540116034
0.9126656520616951
0.9483870399186218
0.9112277195827618
0.8943784544420768
0.9504274192050808
0.9633957881838715
0.9326514732543241
0.8935516261755485
0.9142513467102933
0.9049078534864156
0.930626262015902
0.9144948089965272
0.9514252777195557
0.9216177957532861
0.9598658751925462
0.9485411140583555
0.8864777720709924
0.8564113544394982
0.

In [9]:
diction_lbp = sorted(diction_lbp.items(), key=lambda item:item[1], reverse=True)
print(diction_lbp)

[('carryingShoppingTro', 0.999736564805058), ('carryingUmbrella', 0.9996039081066808), ('upperBodyVNeck', 0.9988107822410148), ('carryingLuggageCase', 0.9761427864304544), ('accessoryKerchief', 0.9697021438057294), ('upperBodyPlaid', 0.9633957881838715), ('lowerBodyHotPants', 0.961361254473908), ('accessoryHeadphone', 0.9611423696451423), ('footwearStocking', 0.9598658751925462), ('carryingBabyBuggy', 0.9597920886206754), ('accessoryHairBand', 0.9596663029427339), ('lowerBodyShortSkirt', 0.9514252777195557), ('upperBodyNoSleeve', 0.9504274192050808), ('upperBodyThinStripes', 0.9485411140583555), ('accessoryMuffler', 0.9483870399186218), ('footwearBoots', 0.943665850146572), ('carryingBackpack', 0.9372817844111028), ('carryingPlasticBags', 0.9326514732543241), ('personalLess15', 0.9308863394326653), ('lowerBodyShorts', 0.930626262015902), ('upperBodyLogo', 0.9268927292090232), ('personalLarger60', 0.9263244041466706), ('accessoryHat', 0.9216369651853522), ('footwearSneakers', 0.92161779

In [10]:
for i in range(61):
    print(accuracy_score(Y_test[:, i], pred_lbp[:, i]))

0.9989473684210526
0.996578947368421
0.9113157894736842
0.911578947368421
0.9581578947368421
0.9821052631578947
0.995
0.9471052631578948
0.9921052631578947
0.9821052631578947
0.9947368421052631
0.9373684210526316
0.9994736842105263
0.9992105263157894
0.9571052631578948
0.9560526315789474
0.9107894736842105
0.9871052631578947
0.9568421052631579
0.9563157894736842
0.9881578947368421
0.9592105263157895
0.9957894736842106
0.9705263157894737
0.9076315789473685
0.9960526315789474
0.9194736842105263
0.9886842105263158
0.9163157894736842
0.9892105263157894
0.9528947368421052
0.9989473684210526
0.9997368421052631
0.9955263157894737
0.9113157894736842
0.9310526315789474
0.9823684210526316
0.9226315789473685
0.9207894736842105
0.9928947368421053
0.9952631578947368
0.9718421052631578
0.9871052631578947
0.9207894736842105
0.9155263157894736
0.991578947368421
0.9618421052631579
0.985
0.9457894736842105
0.9881578947368421
0.9963157894736843
0.9763157894736842
0.9907894736842106
0.9786842105263158
0.9

# HOG

In [8]:
from skimage.feature import hog

data_x = []
for i in range(length):
    image = cv2.imread(data[i, 0])
    image = cv2.resize(image,(160, 75))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fea = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), feature_vector=True)
    data_x.append(fea)
data_x = np.asarray(data_x)
print(data_x.shape)

/home/anhaoran/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


(19000, 320)


In [9]:
from sklearn.neighbors import KNeighborsClassifier

neigh_hog = KNeighborsClassifier(n_neighbors=5)
neigh_hog.fit(X_train, Y_train)
#print(neigh_hog.score(X_test, Y_test))
pred_hog = neigh_hog.predict(X_test)

In [18]:
diction_hog = {}
for i in range(61):
    print( mA(Y_test[:, i], pred_hog[:, i]))#attributes_list[i], ': ',
    diction_hog.update({attributes_list[i]:mA(Y_test[:, i], pred_hog[:, i])})

0.8742063492063492
0.9827826433121019
0.9199383332673972
0.9123022205729445
0.8975185928201652
0.942073325332567
0.9575694802660488
0.9234153327103485
0.9399651650700658
0.9520323804505808
0.916394337467949
0.9134983318052008
0.8998682476943347
0.9059857293868923
0.9009148367113716
0.9059203483124358
0.9066130543214979
0.9143470441664026
0.8993903857787705
0.9060969068478455
0.9488024283871173
0.9244859998367327
0.9806207265154168
0.9121272750742223
0.9085708036495438
0.9668133072005067
0.9127167788203402
0.880885749822317
0.9127206591585031
0.9267857980394916
0.9037263824522407
0.9997368421052631
1.0
0.9834566495921078
0.9066130543214979
0.9164621678008299
0.9474983221113251
0.9123521132118537
0.896914553307465
0.9445851101634684
0.9598161509926216
0.9164092807962538
0.8629162191192266
0.9107277969059056
0.9135655498565733
0.8687219322733342
0.8982768468814317
0.8981060980502753
0.9048968876752931
0.9410022959385639
0.9360403397027601
0.8775562874221343
0.8367626549199098
0.8722443030

In [11]:
diction_hog = sorted(diction_hog.items(), key=lambda item:item[1], reverse=True)
print(diction_hog)

[('lowerBodyThinStripes', 1.0), ('lowerBodyPlaid', 0.9997368421052631), ('upperBodyThickStripes', 0.9985495780590717), ('carryingLuggageCase', 0.9834566495921078), ('personalLess15', 0.9827826433121019), ('lowerBodyHotPants', 0.9806207265154168), ('accessoryKerchief', 0.9668133072005067), ('upperBodyPlaid', 0.9598161509926216), ('carryingBabyBuggy', 0.9575694802660488), ('upperBodySweater', 0.9547822734720025), ('footwearBoots', 0.9520323804505808), ('accessoryHairBand', 0.9488024283871173), ('accessoryMuffler', 0.9474983221113251), ('upperBodyNoSleeve', 0.9445851101634684), ('personalLarger60', 0.942073325332567), ('footwearStocking', 0.9410022959385639), ('hairBald', 0.9399651650700658), ('upperBodyThinStripes', 0.9360403397027601), ('lowerBodyLongSkirt', 0.9267857980394916), ('accessoryHat', 0.9244859998367327), ('carryingBackpack', 0.9234153327103485), ('personalLess30', 0.9199383332673972), ('carryingMessengerBag', 0.9164621678008299), ('carryingPlasticBags', 0.9164092807962538), 

In [12]:
for i in range(61):
    print(accuracy_score(Y_test[:, i], pred_hog[:, i]))

0.9971052631578947
0.996578947368421
0.9181578947368421
0.915
0.96
0.9852631578947368
0.9939473684210526
0.9502631578947368
0.995
0.9860526315789474
0.993421052631579
0.9371052631578948
0.9994736842105263
0.9986842105263158
0.9555263157894737
0.9565789473684211
0.9076315789473685
0.9897368421052631
0.9552631578947368
0.9578947368421052
0.9886842105263158
0.9605263157894737
0.9952631578947368
0.978421052631579
0.9192105263157895
0.9976315789473684
0.9184210526315789
0.9881578947368421
0.9223684210526316
0.991578947368421
0.9536842105263158
0.9994736842105263
1.0
0.9952631578947368
0.9076315789473685
0.93
0.9757894736842105
0.9178947368421052
0.9231578947368421
0.9921052631578947
0.9960526315789474
0.9760526315789474
0.9839473684210527
0.9178947368421052
0.9213157894736842
0.988421052631579
0.9607894736842105
0.9810526315789474
0.9376315789473684
0.9857894736842105
0.9960526315789474
0.9802631578947368
0.988421052631579
0.9773684210526316
0.965
0.9936842105263158
0.9971052631578947
0.908

# GoogLenet

In [13]:
features = np.load("../results/GoogLeNet_binary61_feature_imagenet_all.npy")
X_train_, X_test_, Y_train_ ,Y_test_ = train_test_split(features, data_y, test_size=0.2)

from sklearn.neighbors import KNeighborsClassifier

neigh_goo = KNeighborsClassifier(n_neighbors=5)
neigh_goo.fit(X_train_, Y_train_)
#print(neigh_hog.score(X_test, Y_test))
pred_goo = neigh_goo.predict(X_test_)

In [19]:
diction_goo = {}
for i in range(61):
    print(mA(Y_test_[:, i], pred_goo[:, i]))#attributes_list[i], ': ', 
    diction_goo.update({attributes_list[i]:mA(Y_test_[:, i], pred_goo[:, i])})

0.9992071881606766
0.9863031914893616
0.9716386288551238
0.9697867647452473
0.9760882754029356
0.9927072098232805
0.9839027944945264
0.9554507538202281
0.9934666038970732
0.9807090239410681
0.9833134288747346
0.9497238840312932
0.9998683864174782
0.9996045346691274
0.9773702338157306
0.9813467110639822
0.9635345310305521
0.9696965207884594
0.9773433658259983
0.98101786503679
0.9582724899331829
0.9639169882156637
0.9924214096593507
0.9757110898050496
0.9684790758019364
0.9857142857142858
0.9661583937420416
0.8934346126510305
0.9664398318146203
0.9945046656966288
0.9702567916944846
0.9997368421052631
0.9998684210526316
0.9331472620946306
0.9635289708209556
0.9646871818314044
0.9803592900360929
0.9536375463417092
0.95347924803343
0.9709916251972326
0.9513224713013018
0.9865895846899533
0.9826791536032874
0.9483267996932598
0.9676338403109068
0.9708789740849586
0.9646723442456072
0.9779493511064431
0.947407581453634
0.9808447879181169
0.9080321478415708
0.973318320871331
0.9671835909459672

In [15]:
diction_goo = sorted(diction_goo.items(), key=lambda item:item[1], reverse=True)
print(diction_goo)

[('lowerBodyThinStripes', 0.9998684210526316), ('carryingShoppingTro', 0.9998683864174782), ('lowerBodyPlaid', 0.9997368421052631), ('carryingUmbrella', 0.9996045346691274), ('accessoryHeadphone', 0.9992071881606766), ('lowerBodyLongSkirt', 0.9945046656966288), ('hairBald', 0.9934666038970732), ('personalLarger60', 0.9927072098232805), ('lowerBodyHotPants', 0.9924214096593507), ('carryingPlasticBags', 0.9865895846899533), ('personalLess15', 0.9863031914893616), ('accessoryKerchief', 0.9857142857142858), ('carryingBabyBuggy', 0.9839027944945264), ('lowerBodyCapri', 0.9833134288747346), ('footwearSandals', 0.9826791536032874), ('upperBodyCasual', 0.9813467110639822), ('upperBodyFormal', 0.98101786503679), ('footwearStocking', 0.9808447879181169), ('footwearBoots', 0.9807090239410681), ('accessoryMuffler', 0.9803592900360929), ('lowerBodyShortSkirt', 0.9779493511064431), ('lowerBodyCasual', 0.9773702338157306), ('lowerBodyFormal', 0.9773433658259983), ('personalLess60', 0.9760882754029356

In [16]:
for i in range(61):
    print(accuracy_score(Y_test_[:, i], pred_goo[:, i]))

0.998421052631579
0.9973684210526316
0.973421052631579
0.9702631578947368
0.9871052631578947
0.996578947368421
0.9973684210526316
0.973421052631579
0.9973684210526316
0.993421052631579
0.9976315789473684
0.9623684210526315
0.9997368421052631
0.9992105263157894
0.9886842105263158
0.9876315789473684
0.9639473684210527
0.9931578947368421
0.9889473684210527
0.9886842105263158
0.9910526315789474
0.9826315789473684
0.9986842105263158
0.9857894736842105
0.9736842105263158
0.9997368421052631
0.966842105263158
0.9926315789473684
0.9686842105263158
0.9973684210526316
0.9815789473684211
0.9994736842105263
0.9997368421052631
0.996578947368421
0.9639473684210527
0.9671052631578947
0.9939473684210526
0.9602631578947368
0.9628947368421052
0.9957894736842106
0.9973684210526316
0.9907894736842106
0.9923684210526316
0.9515789473684211
0.9686842105263158
0.9936842105263158
0.9810526315789474
0.9952631578947368
0.9731578947368421
0.9955263157894737
0.9968421052631579
0.9939473684210526
0.9968421052631579


# Group

In [30]:
lbp_better = []
hog_better = []
goo_better = []
for i in range(61):
    att = attributes_list[i]
    if diction_goo[att] - diction_hog[att] >= 0 and diction_goo[att] - diction_lbp[att] >= 0:
        goo_better.append(att)
    if diction_hog[att] - diction_goo[att] >= 0:
        hog_better.append(att)
    if diction_lbp[att] - diction_goo[att] >= 0:
        lbp_better.append(att)

In [31]:
print("LBP better:")
print(lbp_better)
print("HOG better:")
print(hog_better)
print("GoogLeNet better:")
print(goo_better)

LBP better:
['lowerBodyPlaid', 'lowerBodyThinStripes', 'carryingLuggageCase', 'upperBodyPlaid', 'upperBodyThinStripes', 'upperBodyThickStripes', 'upperBodyVNeck']
HOG better:
['lowerBodyPlaid', 'lowerBodyThinStripes', 'carryingLuggageCase', 'upperBodyPlaid', 'upperBodyThinStripes', 'upperBodyThickStripes', 'upperBodyVNeck']
GoogLeNet better:
['accessoryHeadphone', 'personalLess15', 'personalLess30', 'personalLess45', 'personalLess60', 'personalLarger60', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'lowerBodyCapri', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'lowerBodyCasual', 'upperBodyCasual', 'personalFemale', 'carryingFolder', 'lowerBodyFormal', 'upperBodyFormal', 'accessoryHairBand', 'accessoryHat', 'lowerBodyHotPants', 'upperBodyJacket', 'lowerBodyJeans', 'accessoryKerchief', 'footwearLeatherShoes', 'upperBodyLogo', 'hairLong', 'lowerBodyLongSkirt', 'upperBodyLongSleeve', 'lowerBodyPlaid', 'personalMale', 'carryingMessengerBag', 'accessor